# YOUR PROJECT TITLE

# Imports

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
import bqplot as bq
from IPython.display import display
import plotly.graph_objects as go

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Import data sets

In [4]:
# Load the first CSV file
df1 = pd.read_csv('National.csv', sep=";", decimal="," )

# Load the second CSV file
df2 = pd.read_csv('Employ.csv', sep=";", decimal="," )

# Select the desired columns from the first dataset
df1_selected = df1[['Year','Y', 'M', 'X', 'C', 'G', 'I']]

# Select the desired columns from the second dataset
df2_selected = df2[['N']]

# Combine the selected columns into a new dataset
df = pd.concat([df1_selected, df2_selected], axis=1)

# Create a new column by subtracting 'M' from 'X'
df['NX'] = df['X'] - df['M']

# Data exploration

In [5]:
# Create a dropdown menu with the variable names
dropdown = widgets.Dropdown(options=['Y', 'C', 'G', 'I', 'NX', 'N'])

# Create a dictionary that maps the original variable names to the new names
name_dict = {'Y': 'GDP', 'C': 'Consumption', 'G': 'Public consumption', 'I': 'Investment', 'NX': 'Net exports', 'N': 'Employment level'}

# Create a dictionary for the y-axis labels
ylabel_dict = {'Y': 'Billions in DKK', 'C': 'Billions in DKK', 'G': 'Billions in DKK', 'I': 'Billions in DKK', 'NX': 'Billions in DKK', 'N': 'Thousands of people'}

# Create scales for the x and y axes
x_sc = bq.LinearScale(min=2000, max=2024)
y_sc = bq.LinearScale()

# Create a line mark
line = bq.Lines(x=df['Year'], y=[], scales={'x': x_sc, 'y': y_sc})

# Create axes
x_ax = bq.Axis(scale=x_sc)
y_ax = bq.Axis(scale=y_sc, label='', orientation='vertical')

# Create a figure
fig = bq.Figure(marks=[line], axes=[x_ax, y_ax], title='', layout=widgets.Layout(width='600px', height='500px'))

# Create a function to update the figure
def update_figure(change):
    line.y = df[dropdown.value]
    fig.title = f'{name_dict[dropdown.value]} development from 2000 to 2023'
    y_ax.label = ylabel_dict[dropdown.value]

# Update the figure when the dropdown value changes
dropdown.observe(update_figure, 'value')

# Display the dropdown and the figure
display(dropdown)
display(fig)

# Initialize the figure
update_figure(None)

NameError: name 'df_combined' is not defined

# Analysis

In [ ]:
# Convert columns to numeric, excluding 'Year'
for column in df.columns:
    if column != 'Year':
        df[column] = pd.to_numeric(df[column], errors='coerce')

# Create a new DataFrame for the growth rates
df_growth = pd.DataFrame()
df_growth['Year'] = df['Year']

# Calculate the growth rates and add them as new columns, excluding 'Year'
for column in df.columns:
    if column != 'Year':
        df_growth[column + '_growth'] = df[column].pct_change() * 100  # Multiply by 100 to get percentage

# Remove the year 2000 from the DataFrame
df_growth = df_growth[df_growth['Year'] != 2000]

In [ ]:
# Create a bar chart for 'Y_growth'
fig = go.Figure(data=[
    go.Bar(name='Y_growth', x=df_growth['Year'], y=df_growth['Y_growth'])
])

# Add traces for the other variables
variables = ['C_growth', 'G_growth', 'I_growth', 'NX_growth', 'N_growth']
for variable in variables:
    fig.add_trace(
        go.Bar(name=variable, x=df_growth['Year'], y=df_growth[variable], visible=False)
    )

# Create a dropdown menu
buttons = []
for i, variable in enumerate(variables):
    visibility = [True] + [False]*i + [True] + [False]*(len(variables)-i-1)
    buttons.append(dict(label=variable, method='update', args=[{'visible': visibility}, {'title': variable}]))

fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=buttons,
        )
    ],
)

# Set the initial title of the plot
fig.update_layout(title=variables[0])

# Show the figure
fig.show()